In [18]:
import pandas as pd
from utilities import GetPhysQuantities
from wrapper import *
from EoM import GetXi, FriedmannEq
import os
import sys
import optparse
import numpy as np
from scipy.integrate import solve_ivp
from timer import Timer

alpha = 0
Mpl = 1.

class GEF:
    def __init__(x, alpha, beta, Mpl, phi0, dphidt0, M, ntr, SE, approx=False):
        x.alpha = alpha
        x.beta = beta
        x.f = Mpl
        x.mass = M
        x.SE = SE
        x.ini = np.array([phi0, dphidt0])
        if (SE==None):
            x.GaugePos = 4
            x.conductivity = lambda y: 0., 0.
            if(approx):
                x.Whittaker = x.WhittakerApprox_NoSE
            else:
                x.Whittaker = x.WhittakerExact
            #x.ODE = x.fullGEF_NoSE()
        else:
            x.GaugePos = 6
            if (SE=="mix"):
                #x.ODE = x.fullGEF_SE_mixed()
                x.Whittaker = x.WhittakerExact
                x.conductivity = x.ComputeImprovedSigma
            elif (-1. <= SE <=1.):
                #x.ODE = x.fullGEF_SE_collinear(SE)
                x.Whittaker = x.WhittakerExact
                x.conductivity = x.ComputeSigmaCollinear
            else:
                print(SE, "is not a valid choice for SE")
        x.approx = approx
        x.ntr = ntr
        x.omega = np.sqrt((0.5*dphidt0**2 + x.potential(phi0))/(3*Mpl**2))
        x.ratio = x.omega/x.f
    
    #Potentials and Couplings
    def potential(x, phi):
        return 0.5*(phi)**2 * x.mass**2

    def dIdphi(x, phi):
        return x.beta/x.f

    def ddIddphi(x, phi):
        return 0.

    def dVdphi(x, phi):
        return phi * x.mass**2
    
    #Useful Quantities
    def FriedmannEq(x):
        Hsq = (1/3) * (0.5 * x.dphidt**2 + x.a**(2*x.alpha)*
                       (x.V + x.ratio**2*(0.5*(x.E[0]+x.B[0]) + x.rhoChi)))
        x.H = np.sqrt(Hsq)
        return x.H
    
    def GetXi(x):
        x.xi = (x.dI * x.dphidt)/(2 * x.H)
        return x.xi
    
    def GetS(x, sigma):
        return (x.a**(x.alpha) * sigma)/(2* x.H)
    
    def ComputeSigmaCollinear(x):
        mu = ((x.E[0]+x.B[0])/2)**(1/4)
        if mu==0:
             return 0., 0.
        else:
            sign = np.sign(x.G[0])
            mz = 91.2/(1.220932e19)
            gmz = 0.35
            gmu = np.sqrt(gmz**2/(1 + gmz**2*41./(48.*np.pi**2)*np.log(mz/(mu*x.ratio))))

            fracE = (min(1., 1.- x.SE)*x.E[0]/(x.E[0]+x.B[0]) + max(-x.SE, 0.)*x.B[0]/(x.E[0]+x.B[0]))
            x.sigmaE = fracE*(41.*gmu**3/(72.*np.pi**2)
                 * np.sqrt(x.B[0])/(x.H * np.tanh(np.pi*np.sqrt(x.B[0]/x.E[0]))))

            fracB = (min(1., 1.+ x.SE)*x.B[0]/(x.E[0]+x.B[0]) + max(x.SE,0.)*x.E[0]/(x.E[0]+x.B[0]))
            x.sigmaB = fracB * (-sign)*(41.*gmu**3/(72.*np.pi**2)
                 * np.sqrt(x.E[0])/(x.H * np.tanh(np.pi*np.sqrt(x.B[0]/x.E[0]))))

            return x.sigmaE, x.sigmaB

    def ComputeImprovedSigma(x):
        mu = ((x.E[0]+x.B[0])/2)**(1/4)
        if mu==0:
             return 0., 0.
        else:
            mz = 91.2/(1.220932e19)
            gmz = 0.35
            gmu = np.sqrt(gmz**2/(1 + gmz**2*41./(48.*np.pi**2)*np.log(mz/(mu*x.ratio))))

            Sigma = np.sqrt((x.E[0] - x.B[0])**2 + 4*x.G[0]**2)
            Eprime = np.sqrt(x.E[0]-x.B[0]+Sigma)
            Bprime = np.sqrt(x.B[0]-x.E[0]+Sigma)
            Sum = x.E[0] + x.B[0] + Sigma

            sigma = (41.*gmu**3/(72.*np.pi**2)
                     /(np.sqrt(Sigma*Sum)*x.H * np.tanh(np.pi*Bprime/Eprime)))

            x.sigmaE = abs(x.G[0])*Eprime*sigma
            x.sigmaB = -x.G[0]*Bprime*sigma

            return x.sigmaE, x.sigmaB
        
    def EoM(x):
        #Declare all local variables
        a = np.exp(x.y[0])
        
    #Equations of Motions
    def EoMphi(x):
        x.ddphiddt = ((x.alpha-3)*x.H * x.dphidt
                - x.a**(2*x.alpha)*x.dV - x.a**(2*x.alpha)*x.dI*x.G[0]*x.ratio**2)
        return
    
    def EoMlnkh(x):
        alpha = x.alpha
        
        x.xieff = x.xi + x.GetS(x.sigmaB)
        x.s = x.GetS(x.sigmaE)
        x.r = (abs(x.xieff) + np.sqrt(x.xieff**2 + x.s**2 + x.s))
        
        fc = x.a**(1-alpha) * x.H * (x.r)

        dsigmaEdt = 0.
        dsigmaBdt = 0.
        #approximation: dHdt = alphaH**2 (slow-roll)
        fcprime = (((1-alpha)*x.H+x.a**(1-alpha)*alpha*x.H)*fc
                      + x.g(x.xieff)*(x.a**(1-alpha)*(x.ddI*x.dphidt**2 + x.dI*x.ddphiddt)/2
                                           +x.a*(alpha*x.H*x.sigmaB + dsigmaBdt)/2
                                           - x.a**(1-alpha)*alpha*x.H**2*x.xieff)*
                   (1. + x.xieff/np.sqrt(x.xieff**2 + x.s**2 + x.s))
                      + (x.a*(alpha*x.H*x.sigmaE + dsigmaEdt)/2 
                         - x.a**(1-alpha)*alpha*x.H**2*x.s)*(x.g(x.s)*x.s + 1/2)/np.sqrt(x.xieff**2 + x.s**2 + x.s))
        #fprime = f*H
        if (fcprime >= 0):
            if((x.kh-fc)/x.kh <=1e-3):
                dlnkhdt = fcprime/x.kh
            else:
                dlnkhdt = 0
        else:
            dlnkhdt = 0
            
        x.dlnkhdt = dlnkhdt
    
        return
    
    def EoMDelta(x):
        x.ddelta = -x.a**(x.alpha)*x.sigmaE*x.delta
        return
    
    def EoMrhoChi(x):
        x.drhochi = x.a**(x.alpha)*(x.sigmaE*x.E[0] - x.sigmaB*x.G[0])  - 4*x.H*x.rhoChi
        return
    
    def EoMF(x):
        prefac = x.dlnkhdt * x.delta / (4*np.pi**2)
        
        Whitt = x.Whittaker()
        
        Whitt[2,1] = -Whitt[2,1]
        
        bdrF = prefac*np.array([[(x.kh/x.a)**(i+4)*(Whitt[j,0] + (-1)**i*Whitt[j,1]) for j in range(3)]
                                for i in range(x.ntr)])
        
        dFdt = np.zeros(bdrF.shape)
    
        for n in range(x.ntr-1):
            dFdt[n,0] = (bdrF[n, 0] - ((4+n)*x.H + 2*x.a**(x.alpha) * x.sigmaE)*x.E[n]
                         - 2*x.a**(x.alpha)*x.G[n+1] + 2*(x.dI*x.dphidt+x.a**x.alpha*x.sigmaB)*x.G[n])

            dFdt[n,1] = bdrF[n, 1] - ((4+n)*x.H)*x.B[n] + 2*x.a**(x.alpha)*x.G[n+1]

            dFdt[n,2] = (bdrF[n, 2] - ((4+n)*x.H + x.a**(x.alpha) * x.sigmaE)*x.G[n]
                         + x.a**(x.alpha)*(x.E[n+1] - x.B[n+1]) + (x.dI*x.dphidt+x.a**x.alpha*x.sigmaB)*x.B[n])

        dFdt[-1,0] = (bdrF[-1,0] -  ((4+x.ntr)*x.H + 2*x.a**(x.alpha) * x.sigmaE)*x.E[-1]
                        - 2*x.kh**2 * x.a**(x.alpha-2)*x.G[-2]
                          + 2*(x.dI*x.dphidt+x.a**x.alpha*x.sigmaB)*x.G[-1])
        
        dFdt[-1,1] = bdrF[-1,1] - (4+x.ntr)*x.H*x.B[-1] + 2*x.kh**2 * x.a**(x.alpha-2)*x.G[-2]
        
        dFdt[-1,2] = (bdrF[-1,2] - ((4+x.ntr)*x.H + x.a**(x.alpha) * x.sigmaE)*x.G[-1] 
                         + x.kh**2 * x.a**(x.alpha-2)*(x.E[-2] - x.B[-2])
                          + (x.dI*x.dphidt+x.a**x.alpha*x.sigmaB)*x.B[-1])
        x.dE = dFdt[:,0]
        x.dB = dFdt[:,1]
        x.dG = dFdt[:,2]

        return
    
    #Whittaker Functions
    def WhittakerApprox_NoSE(x):
        Fterm = np.zeros((3, 2))
        sgnsort = int((1-np.sign(x.xi))/2)
        Fterm[0, sgnsort] = approxPosE(x.xi)
        Fterm[0, 1-sgnsort] = approxMinE(x.xi)

        Fterm[1, sgnsort] = approxPosB(x.xi)
        Fterm[1, 1-sgnsort] = approxMinB(x.xi)

        Fterm[2, sgnsort] = approxPosG(x.xi)
        Fterm[2, 1-sgnsort] = approxMinG(x.xi)
        """if (abs(x.xi) >= 3):
            Fterm = np.zeros((3, 2))
            sgnsort = int((1-np.sign(x.xi))/2)
            Fterm[0, sgnsort] = approxPosE(x.xi)
            Fterm[0, 1-sgnsort] = approxMinE(x.xi)

            Fterm[1, sgnsort] = approxPosB(x.xi)
            Fterm[1, 1-sgnsort] = approxMinB(x.xi)

            Fterm[2, sgnsort] = approxPosG(x.xi)
            Fterm[2, 1-sgnsort] = approxMinG(x.xi)
        else:
            Fterm = x.WhittakerExact(x)"""
        
        return Fterm
    
    def WhittakerExact(x):
        Whitt1Plus = whitw(-x.xieff*(1j), 1/2 + x.s, -2j*x.r)
        Whitt2Plus = whitw(1-x.xieff*(1j), 1/2 + x.s, -2j*x.r)

        Whitt1Minus = whitw(x.xieff*(1j), 1/2 + x.s, -2j*x.r)
        Whitt2Minus = whitw(1+x.xieff*(1j), 1/2 + x.s, -2j*x.r)
            
        exptermPlus = np.exp(np.pi*x.xieff)
        exptermMinus = np.exp(-np.pi*x.xieff)
        
        Fterm = np.zeros((3, 2))

        Fterm[0,0] = exptermPlus*abs((1j*x.r - 1j*x.xieff -x.s) * Whitt1Plus + Whitt2Plus)**2/x.r**2
        Fterm[0,1] = exptermMinus*abs((1j*x.r + 1j*x.xieff -x.s) * Whitt1Minus + Whitt2Minus)**2/x.r**2

        Fterm[1,0] = exptermPlus*abs(Whitt1Plus)**2
        Fterm[1,1] = exptermMinus*abs(Whitt1Minus)**2

        Fterm[2,0] = exptermPlus*((Whitt2Plus*Whitt1Plus.conjugate()).real - x.s * abs(Whitt1Plus)**2)/x.r
        Fterm[2,1]= exptermMinus*((Whitt2Minus*Whitt1Minus.conjugate()).real - x.s * abs(Whitt1Minus)**2)/x.r

        return Fterm
    
    
    #Auxiliary Functions
    def g(x, val):
            if (val < 0):
                return -1
            elif (val > 0):
                return 1
            else:
                return 0
            
    def approxPosE(xi):
        xi = abs(xi)
        g1 = math.gamma(2/3)**2
        g2 = math.gamma(1/3)**2
        t1 = (3/2)**(1/3)*g1/(np.pi*xi**(1/3))
        t2 = -np.sqrt(3)/(15*xi)
        t3 = (2/3)**(1/3)*g2/(100*np.pi*xi**(5/3))
        t4 = (3/2)**(1/3)*g1/(1575*np.pi*xi**(7/3))
        t5 = -27*np.sqrt(3)/(19250*xi**3)
        t6 = 359*(2/3)**(1/3)*g2/(866250*np.pi*xi**(11/3))
        t7 = 8209*(3/2)**(1/3)*g1/(13162500*np.pi*xi**(13/3))
        t8 = -690978*np.sqrt(3)/(1861234375*xi**5)
        t9 = 13943074*(2/3)**(1/3)*g2/(127566140625*np.pi*xi**(17/3))
        return t1+t2+t3+t4+t5+t6+t7+t8+t9

    def approxMinE(xi):
        t1 = 1
        t2 = -9/(2**(10)*xi**2)
        t3 = 2059/(2**(21)*xi**4)
        t4 = -448157/(2**31*xi**6)
        return np.sqrt(2)*(t1 + t2 + t3 + t4)

    def approxPosB(xi):
        xi = abs(xi)
        g1 = math.gamma(2/3)**2
        g2 = math.gamma(1/3)**2
        t1 = (2/3)**(1/3)*g2*xi**(1/3)/(np.pi)
        t2 = 2*np.sqrt(3)/(35*xi)
        t3 = -4*(2/3)**(1/3)*g2/(225*np.pi*xi**(5/3))
        t4 = 9*(3/2)**(1/3)*g1/(1225*np.pi*xi**(7/3))
        t5 = 132*np.sqrt(3)/(56875*xi**3)
        t6 = -9511*(2/3)**(1/3)*g2/(5457375*np.pi*xi**(11/3))
        t7 = 1448*(3/2)**(1/3)*g1/(1990625*np.pi*xi**(13/3))
        t8 = 1187163*np.sqrt(3)/(1323765625*xi**5)
        t9 = -22862986*(2/3)**(1/3)*g2/(28465171875*np.pi*xi**(17/3))
        return t1+t2+t3+t4+t5+t6+t7+t8+t9

    def approxMinB(xi):
        t1 = 1
        t2 = 11/(2**(10)*xi**2)
        t3 = -2397/(2**(21)*xi**4)
        t4 = 508063/(2**31*xi**6)
        return 1/np.sqrt(2)*(t1 + t2 + t3 + t4)

    def approxPosG(xi):
        xi = abs(xi)
        g1 = math.gamma(2/3)**2
        g2 = math.gamma(1/3)**2
        t1 = 1/np.sqrt(3)
        t2 = -(2/3)**(1/3)*g2/(10*np.pi*xi**(2/3))
        t3 = 3*(3/2)**(1/3)*g1/(35*np.pi*xi**(4/3))
        t4 = -np.sqrt(3)/(175*xi**2)
        t5 = -41*(2/3)**(1/3)*g2/(34650*np.pi*xi**(8/3))
        t6 = 10201*(3/2)**(1/3)*g1/(2388750*np.pi*xi**(10/3))
        t7 = -8787*np.sqrt(3)/(21896875*xi**4)
        t8 = -1927529*(2/3)**(1/3)*g2/(4638768750*np.pi*xi**(14/3))
        t9 = 585443081*(3/2)**(1/3)*g1/(393158390625*np.pi*xi**(16/3))
        t10 = -65977497*np.sqrt(3)/(495088343750*xi**6)
        return t1+t2+t3+t4+t5+t6+t7+t8+t9+t10

    def approxMinG(xi):
        t1 = 1
        t2 = -67/(2**(10)*xi**2)
        t3 = 21543/(2**(21)*xi**4)
        t4 = -6003491/(2**31*xi**6)
        return -np.sqrt(2)/(32*abs(xi))*(t1 + t2 + t3 + t4)       
            
    #Run GEF
    def InitialiseGEF(x):
        yini = np.zeros((x.ntr*3+x.GaugePos))
        yini[0] = 0
        yini[1] = x.ini[0]/x.f
        yini[2] = x.ini[1]/(x.f*x.omega)
        yini[3] = np.log(abs(yini[2]*x.beta))
        if (x.SE != None):
            yini[4] = Delta0
            yini[5] = rhoChi0
        #print(yini)
        return yini
    
    def TimeStep(x, t, y):
        x.DefineVariables(t, y)
        dydt = np.zeros(y.shape)
        dydt[0] = x.H
        dydt[1] = x.dphidt
        x.EoMphi()
        dydt[2] = x.ddphiddt
        x.EoMlnkh()
        dydt[3] = x.dlnkhdt

        if (x.SE != None):
            dydt[4] = x.ddelta
            x.EoMrhoChi()
            dydt[5] = x.drhoChi

        x.EoMF()
        dFdt = np.array([x.dE, x.dB, x.dG]).T
        dydt[x.GaugePos:] = dFdt.reshape(x.ntr*3)
        return dydt
    
    def SolveGEF(x):
        t = Timer()
        yini = x.InitialiseGEF()
        ODE = lambda t, y: x.TimeStep(t, y)
        t.start()
        sol = solve_ivp(ODE, [0,120], yini, method="RK45")
        t.stop()
        return sol
    
    def DefineVariables(x, t, y):
        f = x.f
        omega = x.omega
        x.t = t
        x.N = y[0]
        x.a = np.exp(x.N)
        
        x.phi = y[1]
        x.dphidt = y[2]
        x.kh = np.exp(y[3])
        
        x.V = x.potential(f*x.phi)/(omega*f)**2
        x.dV = x.dVdphi(f*x.phi)/(f*omega**2)
        x.dI = x.dIdphi(f*x.phi)*f
        x.ddI = x.ddIddphi(f*x.phi)*f**2
        
        F = y[x.GaugePos:]
        F = F.reshape(x.ntr, 3)
        x.E = F[:,0]
        x.B = F[:,1]
        x.G = F[:,2]
        
        if (x.SE == None):
            x.rhoChi = 0.
            x.delta = 1.
            x.FriedmannEq()
            #print(x.H)
            x.sigmaE = 0.
            x.sigmaB = 0.
        else:
            x.delta = y[4]
            x.rhoChi = y[5]
            x.FriedmannEq()
            x.conductivity()
        
        x.xi = x.GetXi()
        

In [19]:
alpha = 0
beta = 25.
Mpl = 1
M = 6e-6*Mpl
phi0 = 15.55*Mpl #in Pl units
dphidt0 = -np.sqrt(2/3)*M*Mpl
ntr = 56
SE = None
G = GEF(alpha, beta, Mpl, phi0, dphidt0, M, ntr, SE, approx=True)

In [20]:
sol = G.SolveGEF()

Elapsed time: 9.9746 seconds


In [ ]:
G = GEF(alpha, beta, Mpl, phi0, dphidt0, M, ntr, SE, approx=False)

In [ ]:
sol = G.SolveGEF()